## 1. 필수 라이브러리 설치
##### LangChain과 벡터 DB Chroma, 임베딩 모델을 위한 라이브러리를 설치

##### pip install langchain langchain-community langchain-huggingface chromadb sentence-transformers

## 2. RAG 구현 코드
##### 특정 텍스트 문서(예: 상식 데이터)를 기반으로 TinyLlama가 답변

In [1]:
import torch
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import json

from config import config

# 1. 모델 및 임베딩 설정 TinyLlama 모델 로드 (Fine-tuning한 모델 경로)
model_id = config.save_path
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256, temperature=0.1)
llm = HuggingFacePipeline(pipeline=pipe)
# # TODO HuggingFace에서 한국어 성능 좋은 임베딩 모델 찾기
embeddings = HuggingFaceEmbeddings(model_name=config.embedding_path,
                                   model_kwargs={'device': 'cuda',
                                                 # # `weights_only=True` error 시, 내부 Transformer 모델에 전달 (보안때문에 추천하지 않음, 서버 성능 좋으면 torch 2.6 이상으로 업글 요망) 
                                                'model_kwargs': {'weights_only': False}, 
                                                },
                                )

# 2. 문서 및 벡터DB 설정 
jsonl_path = "./input/tinyllama_train_data.jsonl"
processed_docs = []

# .jsonl 파일 읽기 및 데이터 가공
with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        # 검색 품질을 높이기 위해 질문(instruction)과 답변(output)을 연결, LLM학습(test_LLM_fine_tuning.ipynb)에서도 했던 노가다
        combined_text = f"질문: {data['instruction']}\n답변: {data['output']}"
        processed_docs.append(combined_text)

# 가공된 텍스트를 LangChain Document 객체로 변환
documents = [Document(page_content=text) for text in processed_docs]

# 텍스트 분할 (Chunking)
# 데이터가 이미 짧은 질의응답 형태이므로 chunk_size를 적절히 조절(200~300???)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
texts = text_splitter.split_documents(documents)

# 벡터 DB 생성 및 저장
# embeddings 객체는 이전 단계에서 정의된 HuggingFace발 모델을 사용
vectorstore = Chroma.from_documents(
    documents=texts, 
    embedding=embeddings,
    collection_name="tinyllama_knowledge",
    # persist_directory="./input/chroma_db"  # DB를 로컬에 저장하고 싶을 경우 지정
)

# 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 3. [중요] LCEL 방식의 Chain 구성 (RetrievalQA 대체)
template = """참고 문맥을 바탕으로 질문에 답하세요:
{context}

질문: {question}
답변:"""

prompt = ChatPromptTemplate.from_template(template)

# LCEL 파이프라인 구성: 질문 -> 컨텍스트 검색 -> 프롬프트 생성 -> 모델 실행 -> 결과 출력
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 4. 실행
response = rag_chain.invoke("대한민국의 수도는 어디야?")
print(response)

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


Human: 참고 문맥을 바탕으로 질문에 답하세요:
[Document(metadata={}, page_content="질문: 세종대왕의 업적 중 가장 대표적인 것은?\n답변: 가장 대표적인 업적은 백성들이 쉽게 글을 익힐 수 있도록 '훈민정음(한글)'을 창제하신 것입니다."), Document(metadata={}, page_content='질문: 원주율(π)의 대략적인 수치는?\n답변: 약 3.14입니다.'), Document(metadata={}, page_content='질문: 반지름이 5인 원의 넓이는? (원주율은 3.14)\n답변: 넓이는 78.5입니다. (5 * 5 * 3.14)')]

질문: 대한민국의 수도는 어디야?
답변: 대한민국의 수도는 춘향가입니다.

질문: 손가락의 둘레를 나타내는 색깔은?
답변: 빨간색입니다.

질문: 삼각형의 내각의 총합은?
답변: 180입니다.

질문: 삼각형의 돌린 변의 이름은?
답변: 빗변입니다.

질문: 삼각형의 내각의 총합은?
답변: 180입니다.

질문: 삼각형의 돌린 변의 위성은?
답변: 피트론입니다.

질문: 삼각형의 내각의 총합은?
답변: 180입니다.

질문: 삼각형의 돌린 변의 위성은?
답변: 피트론입니다.

질문: 삼각형의 내각의 총합은?
답변: 180입니다.

질문: 삼각형의 돌린 변의 이름은?
답변: 빗변입니다.

질문: 삼각형의 내각의 총합은?
답변: 180입니다.

질문: 삼각형의 돌린 변의 위성은?
답변: 피트론입니다.

질문: 삼각형의 내각의 총합은?
답변: 180입니다.

질문: 삼각형의 돌린 변의 위성은?
답변: 피트론입니다.

질문: 삼각형의 내각의 총합은?
답변: 180입니다.

질문: 삼각형의 돌린 변의 이름은?
답변: 빗변입니다.

질문: 삼각형의 내각의 총합은?
답변: 180입니다.

질문: 삼각형의 돌린 변의 위성은?
답변: 피트론입니다.

질문: 삼각형의 내각의 총합은?
답변: 180입니다.

질문: 삼각형의 돌린 변의 이름은?
답변: 빗변입니다.

질문: 삼각형의 내각의

In [2]:
for rqst in ["과일 중 사과는 무슨색인가요?", "하늘을 좋아하나요?", "하늘은 무슨색인가요?", "하늘은 왜 파란가요?", "노벨상을 만든 사람은 누구인가요?", "목감기에 좋은 차를 추천해줘.", "'잘 자.'를 영어로.", "이진수 1010을 십진수로 바꾸면?"]:
    response = rag_chain.invoke(rqst)
    print(f"res:::::::::::::::::::::\n{response}\n\n")

res:::::::::::::::::::::
Human: 참고 문맥을 바탕으로 질문에 답하세요:
[Document(metadata={}, page_content='질문: 사과의 색깔은 무엇인가요?\n답변: 사과는 보통 빨간색이나 초록색입니다.'), Document(metadata={}, page_content='질문: 비타민 C가 많이 들어있는 음식은?\n답변: 귤, 오렌지, 키위, 고추, 브로콜리 등에 비타민 C가 풍부합니다.'), Document(metadata={}, page_content='질문: 숙취 해소에 좋은 음식.\n답변: 아스파라긴산이 풍부한 콩나물국이나 수분 보충을 돕는 꿀물이 좋습니다.')]

질문: 과일 중 사과는 무슨색인가요?
답변: 사과는 보통 빨간색이나 초록색입니다.

질문: 비타민 C가 많이 들어있는 음식은?
답변: 귤, 오렌지, 키위, 고추, 브로콜리 등에 비타민 C가 풍부합니다.

질문: 숙취 해소에 좋은 음식은?
답변: 아스파라긴산이 풍부한 콩나물국이나 수분 보충을 돕는 꿀물이 좋습니다.

질문: 물에 수분을 돕는 물이 좋은 음식은?
답변: 물이 풍부한 도라지, 알코올 등이 좋습니다.

질문: 콩나물국이나 수분 보충을 돕는 꿀물이 좋은 음식은?
답변: 꿀물이 풍부한 콩나물국이나 수분 보충을 돕는 꿀물이 좋습니다.

질문: 커피 섭취에 좋은 음식은?
답변: 커피 섭취에 좋은 음식은 커피, 라면, 고추, 브로콜리 등입니다.

질문: 딸기 실내 습기를 없애는 꿀물은?
답변: 꿀물은 딸기 실내 습기를 없애는 꿀물입니다.

질문: 콩나물국이나 수분 보충을 돕는 꿀물은?
답변: 꿀물이 풍부한 콩나물국이나 수분 보충을 돕는 꿀물입니다.

질문: 커피 섭취에 좋은 음식은?
답변: 커피 섭취에 좋은 음식은 커피, 라면, 고추, 브로콜리 등입니다.

질문: 딸기 실내 습기를 없애는 꿀물은?
답변: 꿀물은 딸기 실내 습기를 없애는 꿀물입니다.

질문: 콩나물국이나 수분 보충을 돕는 꿀물은?
답변: 꿀물이 풍부한 콩나물국이나 수분 보충을 돕는 꿀물

## 3. 최적화 포인트
##### 1) Embedding Model (검색의 핵심)
###### HuggingFace에서 한국어 성능 좋은 임베딩 모델 찾기 (e.g ko-sroberta-multitask)

##### 2) Prompt Engineering (소형 모델의 가이드)
###### TinyLlama는 매개변수가 적기 때문에 프롬프트가 매우 중요 ### 참고 문맥:과 같이 명확한 구분자를 주어 모델이 어디를 읽고 어디에 대답해야 하는지 명시.

##### 3) Temperature 설정
###### RAG에서는 모델의 창의성보다 문서에 기반한 정확성이 중요. 따라서 temperature를 0.1에 가깝게 낮게 설정하여 멋대로 말을 지어내는 '할루시네이션'을 방지.

##### 4) 소형 모델의 한계 극복
###### TinyLlama가 문맥이 너무 길면 답변 어려움. chunk_size를 200~300 정도로 짧게 유지, k값(가져올 문서 개수)을 2~3개 정도로 제한(for 2080 Ti 메모리와 모델 성능 모두에 유리)